In [1]:
import pandas
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
path = sys.path.insert(1,"C:/PythonEnviroments/pytorch_resnet_cifar10")
from resnet import resnet110
import dill  # in order to save Lambda Layer

In [2]:
# the network architecture coresponding to the checkpoint
model = resnet110()

# remember to set map_location
check_point = torch.load('models/resnet110-1d1ed7c2.pth', map_location='cuda:0')

# cause the model are saved from Parallel, we need to wrap it
model = torch.nn.DataParallel(model)
model.load_state_dict(check_point['state_dict'])

# pay attention to .module! without this, if you load the model, it will be attached with [Parallel.module]
# that will lead to some trouble!
torch.save(model.module, 'resnet20_check_point.pth', pickle_module=dill)

# load the converted pretrained model
net = torch.load('resnet20_check_point.pth', map_location='cuda:0')


In [3]:

import torchvision.transforms as transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
transform_train = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

train_set = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True,
    transform=transform_train,
)
test_set = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True,
    transform=transform_test
)


train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=128, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
# accuracy
net.eval()
correct = 0
total = 0
for i, (images, labels) in enumerate(test_loader):
    images = images.cuda()
    labels = labels.cuda()
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()



In [5]:
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the 10000 test images: 93 %
